
# Batch data processing & querying

Importing the integration notebook.

In [ ]:
%run /Users/<user_email>/AWS_S3_mounting_Notebook

## Data ingestion

Reading the data from the pin topic into a table and checking it's content.

In [ ]:
df_pin = make_df_tables('df_pin', 'pin')
df_pin.show(5)

+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+---------------------+--------------------+
 category| description|downloaded|follower_count| image_src|index|is_image_or_video| poster_name| save_location| tag_list| title| unique_id|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+---------------------+--------------------+
 home-decor|Традиционные швед...| 1| 136k|https://i.pinimg....| 6717| image|PUFIK Interiors &...|Local save in /da...|Scandinavian Cott...|〚 Уютные шведские...|bc5ab9ee-505e-44f...|
 christmas|Features: Materia...| 1| 5k|https://i.pinimg....| 1706| image| Wear24-7|Local save in /da...|Merry Christmas T...| Standing Figurine...|b5c8a1b5-9e90-452...|
 christmas|Christmas decorat...| 1| 46k|https://i.pinimg....| 2482| video|Life on Summerhil...|Local save in /da...|Diy Christmas Dec...| FORNT PORCH CHRIS...|08604f20-fa17-4b9...|
 art|Marble Wall Art M...| 1| 305|https://i.pinimg....| 527| image| Wall Canvas Mall|Local save in /da...|Modern Art Painti...| Blue Gold Marble ...|ed8af037-ee87-4a8...|
event-planning|Updated: January ...| 1| 4k|https://i.pinimg....| 4357| image|EventPlanning.com...|Local save in /da...|Event Planning Qu...| First Steps in La...|ccf116e9-9096-494...|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+---------------------+--------------------+
only showing top 5 rows


Reading the data from the geo topic into a table and checking it's content.

In [ ]:
df_geo = make_df_tables('df_geo', 'geo')
df_geo.show(5)

+--------------------+----+--------+---------+-------------------+
 country| ind|latitude|longitude| timestamp|
+--------------------+----+--------+---------+-------------------+
Antarctica (the t...|2418|-88.4642| -171.061|2022-05-27T11:30:59|
Antarctica (the t...|2418|-88.4642| -171.061|2022-05-27T11:30:59|
Antarctica (the t...|5162|-71.6607| -149.206|2019-09-27T19:06:43|
Antarctica (the t...|1335|-77.9931| -175.682|2022-03-19T17:29:42|
Saint Vincent and...|2301| 13.4683| 51.7244|2020-11-14T00:25:28|
+--------------------+----+--------+---------+-------------------+
only showing top 5 rows


Reading the data from the user topic into a table and checking it's content.

In [ ]:
df_user = make_df_tables('df_user', 'user')
df_user.show(5)

+---+-------------------+-----------+-----+---------+
age| date_joined| first_name| ind|last_name|
+---+-------------------+-----------+-----+---------+
 27|2016-03-08T13:38:37|Christopher| 2015| Bradshaw|
 59|2017-05-12T21:22:17| Alexander|10673|Cervantes|
 48|2016-02-27T16:57:44|Christopher| 1857| Hamilton|
 39|2016-06-29T20:43:59| Christina| 6398|Davenport|
 20|2015-10-23T04:13:23| Alexandria| 3599| Alvarado|
+---+-------------------+-----------+-----+---------+
only showing top 5 rows

## Data cleaning

### 1. Cleaning the pinterest posts dataset

In [ ]:
# In a dictionary each key has to be unique, hence the value to replace is the key and the column is the value
# due to the description column appears three times.    
to_replace = {
    'No description available': 'description',
    'Image src error.': 'image_src',
    'User Info Error': 'poster_name',
    'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e': 'tag_list', 
    'No Title Data Available': 'title',
    'No description available Story format': 'description',
    'Untitled': 'description'
}

for value, column in to_replace.items():
    df_pin = df_pin.replace({value: None}, subset=[column])

# There is an extra cleaning necessary as it would be a duplicate in the dictionary.
df_pin = df_pin.replace({'User Info Error': None}, subset=['follower_count'])

df_pin = df_pin.dropDuplicates(df_pin.columns)

In [ ]:
# Rewriting the follower_count coliumn to be numeric only.
df_pin = df_pin.withColumn('follower_count', regexp_replace(df_pin['follower_count'], 'k','000'))
df_pin = df_pin.withColumn('follower_count', regexp_replace(df_pin['follower_count'], 'M','000000'))

# Transforming the column to integer format.
df_pin = df_pin.withColumn('follower_count', df_pin['follower_count'].cast('int'))

# Casting column into numeric types
df_pin = df_pin.withColumn('downloaded', df_pin['downloaded'].cast('int'))
df_pin = df_pin.withColumn('index', df_pin['index'].cast('int'))

# Cleaning the save_location column
df_pin = df_pin.withColumn('save_location', regexp_replace(df_pin['save_location'], 'Local save in ',''))

# Renaming index column
df_pin = df_pin.withColumnRenamed('index', 'ind')

# Rearranging the columns
df_pin = df_pin.select('ind',
                    'unique_id',
                    'title',
                    'description',
                    'follower_count',
                    'poster_name',
                    'tag_list',
                    'is_image_or_video',
                    'image_src',
                    'save_location',
                    'category'
)


### Cleaning the geolocation dataset

In [ ]:
# Creating a new column coordinates with the latitude and longitude as array.
df_geo = df_geo.withColumn('coordinates', array(df_geo['latitude'], df_geo['longitude']))

# Dropping the latutide and longitude columns as no longer needed.
df_geo = df_geo.drop('latitude','longitude')

# Casting the timestamp column into timestamp format.
df_geo = df_geo.withColumn('timestamp', to_timestamp('timestamp'))
df_geo = df_geo.withColumn('timestamp', date_format('timestamp', 'yyyy-MM-dd HH:mm:ss'))

# Dropping duplicate entries
df_pin = df_pin.dropDuplicates(df_pin.columns)

# Reordering the columns
df_geo = df_geo.select('ind',
                       'country',
                       'coordinates',
                       'timestamp')


### Cleaning the users dataset

In [ ]:
# Creating a new colum containing the first name and last name of the user
df_user = df_user.withColumn('user_name', concat(df_user['first_name'], lit(' '), df_user['last_name']))

# Dropping the first_name and last_name columns as no longer needed.
df_user = df_user.drop('first_name','last_name')

# Casting the date_joined column into timestamp format.
df_user = df_user.withColumn('date_joined', to_timestamp('date_joined'))
df_user = df_user.withColumn('date_joined', date_format('date_joined', 'yyyy-MM-dd HH:mm:ss'))

# Dropping duplicate entries
df_pin = df_pin.dropDuplicates(df_pin.columns)

# Reordering the columns
df_user = df_user.select('ind',
                       'user_name',
                       'age',
                       'date_joined')


## Data querying

#### 1. What are the most popular Pinterest category people post to based on their country?

In [ ]:
import pyspark.sql.functions as F

df_joined = df_pin.join(df_geo, df_pin['ind'] == df_geo['ind'], how = 'inner')

In [ ]:
q1_temp = df_joined.groupBy('country','category')\
              .agg(count("category")\
              .alias("category_count"))\
              .orderBy(['category_count'], ascending=False)
w = Window.partitionBy('country')
q1 = q1_temp.withColumn('max_count', F.max('category_count').over(w))\
           .where(F.col('category_count') == F.col('max_count'))\
           .drop('max_count')
display(q1)

country,category,category_count
Afghanistan,education,3
Albania,art,2
Albania,mens-fashion,2
Albania,beauty,2
Algeria,quotes,6
American Samoa,beauty,2
American Samoa,education,2
American Samoa,tattoos,2
Andorra,quotes,2
Angola,diy-and-crafts,2


#### 2. How many posts each category had between 2018 and 2022?

In [ ]:
df_joined = df_joined.withColumn('post_year', year(df_joined['timestamp']))
q2 =   df_joined.filter("post_year BETWEEN '2018' AND '2022'")\
                .groupBy('post_year','category')\
                .agg(count('*')\
                .alias('category_count'))\
                .orderBy(['post_year', 'category_count'], ascending=False)
display(q2)

post_year,category,category_count
2022,beauty,8
2022,christmas,7
2022,quotes,6
2022,diy-and-crafts,5
2022,tattoos,4
2022,mens-fashion,4
2022,home-decor,4
2022,art,3
2022,vehicles,3
2022,education,2


#### 3. Who are the most followed users in each country? 

In [ ]:
df_joined = df_pin.join(df_geo, df_pin['ind'] == df_geo['ind'], how = 'inner')\
                  .join(df_user, df_pin['ind'] == df_user['ind'], how = 'inner')
df_joined = df_joined.dropDuplicates(df_joined.columns)

In [ ]:
w = Window.partitionBy('country')
q3_step1 = df_joined.withColumn('max_follower', F.max('follower_count').over(w))\
                   .where(F.col('follower_count') == F.col('max_follower'))\
                   .drop('max_follower')\
                   .select(['country', 'user_name', 'follower_count'])\
                   .dropDuplicates(q_step1.columns)
display(q3_step1)

country,user_name,follower_count
Afghanistan,Amanda Carlson,3000000
Albania,Aaron Anderson,5000000
Algeria,Aaron Abbott,942000
American Samoa,Abigail Bates,8000000
Andorra,Alison Bell,1000000
Angola,April Brown,8000000
Anguilla,Corey Andrews,92000
Antarctica (the territory South of 60 deg S),Benjamin Campbell,1000000
Antigua and Barbuda,Ann Chung,1000000
Argentina,Andrew Anderson,800000


In [ ]:
q3_step2 = q_step1.groupBy('country')\
                 .agg(max('follower_count')\
                 .alias('follower_count'))\
                 .orderBy(['follower_count'], ascending=False)
display(q3_step2)

country,follower_count
American Samoa,8000000
Angola,8000000
Azerbaijan,6000000
Bouvet Island (Bouvetoya),5000000
Albania,5000000
Bangladesh,4000000
Afghanistan,3000000
Christmas Island,3000000
Antigua and Barbuda,1000000
Martinique,1000000


#### 4. What are the most popular categories by age group?

In [ ]:
df_joined = df_joined.withColumn('age_group',
             F.when(df_joined['age'] <= 24, '18-24')
              .when(df_joined['age'] <= 35, '25-35')
              .when(df_joined['age'] <= 50, '36-50')
              .otherwise('50+'))

q4_temp = df_joined.groupBy('age_group','category')\
              .agg(count('*')\
              .alias('category_count'))\
              .orderBy('age_group')

w = Window.partitionBy('age_group')
q4 = q4_temp.withColumn('max_count', F.max('category_count').over(w))\
           .where(F.col('category_count') == F.col('max_count'))\
           .drop('max_count')

display(q4)

age_group,category,category_count
18-24,art,14
25-35,finance,11
36-50,quotes,8
50+,beauty,3
50+,education,3
50+,vehicles,3


#### 5. What is the median follower count for users by age group?

In [ ]:
from pyspark.sql import functions as F

q5 = df_joined.groupBy('age_group')\
              .agg(percentile_approx('follower_count', 0.5)\
              .alias('median_follower_count'))
display(q5)


age_group,median_follower_count
50+,3000
36-50,6000
18-24,60000
25-35,31000


#### 6. What are the number of users joined each year?

In [ ]:
df_joined = df_joined.withColumn('join_year', year(df_joined['date_joined']))
q6 = df_joined.filter("join_year BETWEEN '2015' AND '2020'")\
              .groupBy('join_year')\
              .agg(count_distinct('unique_id','join_year'))\
              .alias('number_users_joined')
display(q6)

join_year,"count(unique_id, join_year)"
2015,98
2016,132
2017,47


#### 7. What us the median follower count of users based on their joining year?

In [ ]:
q7 = df_joined.filter("join_year BETWEEN '2015' AND '2020'")\
              .groupBy('join_year')\
              .agg(percentile_approx('follower_count', 0.5)\
              .alias('median_follower_count'))
display(q7)

join_year,median_follower_count
2015,67000
2016,23000
2017,6000


#### 8. What are the median follower count of users based on their joining year and age group?

In [ ]:
q8 = df_joined.filter("join_year BETWEEN '2015' AND '2020'")\
              .groupBy('age_group','join_year')\
              .agg(percentile_approx('follower_count', 0.5)\
              .alias('median_follower_count'))\
              .orderBy('age_group','join_year')
display(q8)

age_group,join_year,median_follower_count
18-24,2015,211000
18-24,2016,40000
18-24,2017,11000
25-35,2015,42000
25-35,2016,27000
25-35,2017,8000
36-50,2015,6000
36-50,2016,9000
36-50,2017,3000
50+,2015,196
